In [1]:
import os
import numpy as np
import pandas as pd
import librosa

def calculate_frequency_category(freq):
    # Define frequency category ranges
    categories = [
        (20, 40), (40, 80), (80, 160), (160, 300), (300, 600),
        (600, 1200), (1200, 2400), (2400, 5000), (5000, 10000),
        (10000, 20000), (20000, np.inf)
    ]
    
    # Determine frequency category based on input frequency
    for i, (low, high) in enumerate(categories):
        if low <= freq < high:
            return f'{low}-{high}'  # Return category range as string
    return '20000-inf'  # For frequencies > 20 kHz

def extract_frequency_features(audio_file, duration=5, sample_rate=44100):
    # Load audio file
    y, sr = librosa.load(audio_file, sr=sample_rate)
    
    # Split into sub-files of specified duration
    sub_files = librosa.effects.split(y, top_db=20, frame_length=duration*sample_rate, hop_length=duration*sample_rate)
    
    features = []
    
    for start, end in sub_files:
        sub_y = y[start:end]
        
        # Compute STFT
        D = librosa.stft(sub_y)
        
        # Compute magnitudes
        mag = np.abs(D)
        
        # Calculate average magnitude across time frames
        avg_mag = np.mean(mag, axis=1)
        
        # Calculate dB values
        db_values = librosa.amplitude_to_db(avg_mag)
        
        # Categorize dB values into frequency bands
        freq_bins = librosa.fft_frequencies(sr=sr)
        freq_categories = [calculate_frequency_category(freq) for freq in freq_bins]
        
        # Create dictionary to store category dB values
        category_dbs = {category: [] for category in set(freq_categories)}
        
        for i in range(len(freq_bins)):
            category = freq_categories[i]
            category_dbs[category].append(db_values[i])
        
        # Take average of dB values for each category
        category_avg_dbs = {category: np.mean(values) for category, values in category_dbs.items()}
        
        # Append to features list
        features.append(category_avg_dbs)
    
    return features

def load_audio_files(directory):
    """
    Load audio files from a directory.
    
    Parameters:
        directory (str): Path to the directory containing audio files.
    
    Returns:
        List of audio file paths.
    """
    audio_files = []
    
    # Iterate over files in the directory
    for file in os.listdir(directory):
        if file.endswith('.wav'):  # Assuming audio files are in WAV format
            audio_files.append(os.path.join(directory, file))
    
    return audio_files

def create_dataset(unmastered_dir, mastered_dir):
    """
    Create a dataset containing frequency category dB values for both unmastered and mastered audio files.
    
    Parameters:
        unmastered_dir (str): Path to the directory containing unmastered audio files.
        mastered_dir (str): Path to the directory containing mastered audio files.
    
    Returns:
        DataFrame containing frequency category dB values along with file names for each audio segment.
    """
    unmastered_files = load_audio_files(unmastered_dir)
    mastered_files = load_audio_files(mastered_dir)
    
    data = []
    
    # Process unmastered and mastered audio files together
    for unmastered_file, mastered_file in zip(unmastered_files, mastered_files):
        unmastered_features = extract_frequency_features(unmastered_file)
        mastered_features = extract_frequency_features(mastered_file)
        
        # Combine features from unmastered and mastered files into the same row
        for i in range(min(len(unmastered_features), len(mastered_features))):
            row = {'unmastered_audio_file': os.path.basename(unmastered_file), 'mastered_audio_file': os.path.basename(mastered_file)}
            
            # Add unmastered dB values for each category range
            for category_range, db_value in sorted(unmastered_features[i].items()):
                row[f'unmastered_{category_range}'] = db_value
            
            # Add mastered dB values for each category range
            for category_range, db_value in sorted(mastered_features[i].items()):
                row[f'mastered_{category_range}'] = db_value
            
            data.append(row)
    
    # Create DataFrame from collected data
    df = pd.DataFrame(data)
    
    # Reorder columns based on frequency category ranges
    category_ranges = [
        f'{low}-{high}' for (low, high) in [
            (20, 40), (40, 80), (80, 160), (160, 300), (300, 600),
            (600, 1200), (1200, 2400), (2400, 5000), (5000, 10000),
            (10000, 20000), (20000, np.inf)
        ]
    ]
    
    # Create a list of column names in the desired order
    column_order = []
    for prefix in ['unmastered_', 'mastered_']:
        for category_range in category_ranges:
            column_order.append(f'{prefix}{category_range}')
    
    # Add file name columns at the beginning of the DataFrame
    column_order = ['unmastered_audio_file', 'mastered_audio_file'] + column_order
    
    # Rearrange columns in DataFrame
    df = df[column_order]
    
    return df

# Specify directories containing unmastered and mastered audio files
unmastered_dir = 'drums_unmastered'
mastered_dir = 'drums_mastered'

# Create dataset DataFrame
dataset = create_dataset(unmastered_dir, mastered_dir)

# Save dataset to CSV
dataset.to_csv('audio_features_drums.csv', index=False)


In [2]:
import pandas as pd

# Load the CSV file into a DataFrame
input_csv_path = 'audio_features_drums.csv'
df = pd.read_csv(input_csv_path)

# Define the frequency categories to remove
categories_to_remove = ['<20-40', '20000-inf']  # Adjust as needed based on your category naming convention

# Generate a list of columns to drop
columns_to_drop = []
for category in categories_to_remove:
    columns_to_drop.extend([f'unmastered_{category}', f'mastered_{category}'])

# Drop the specified columns from the DataFrame
df_cleaned = df.drop(columns=columns_to_drop, errors='ignore')

# Save the cleaned DataFrame to a new CSV file
output_csv_path = 'output_dataset_drums.csv'
df_cleaned.to_csv(output_csv_path, index=False)

print(f"Cleaned dataset saved to: {output_csv_path}")


Cleaned dataset saved to: output_dataset_drums.csv


In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split


In [2]:
# Load your dataset (assumed to be in a CSV file)
dataset_path = 'output_dataset_drums.csv'
df = pd.read_csv(dataset_path)

# Extract features (gain values for each frequency range) and target variables
X = df[['unmastered_20-40', 'unmastered_40-80', 'unmastered_80-160','unmastered_160-300','unmastered_300-600','unmastered_600-1200','unmastered_1200-2400','unmastered_2400-5000','unmastered_5000-10000','unmastered_10000-20000']].values
y = df[['mastered_20-40', 'mastered_40-80', 'mastered_80-160','mastered_160-300','mastered_300-600','mastered_600-1200','mastered_1200-2400','mastered_2400-5000','mastered_5000-10000','mastered_10000-20000']].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [3]:
# Define the neural network architecture
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(10,)),  # Input layer with 10 features
    keras.layers.Dense(32, activation='relu'),  # Hidden layer with 32 neurons
    keras.layers.Dense(10)  # Output layer with 10 neurons (one for each frequency range)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Print the model summary
model.summary()




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                704       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 10)                330       
                                                                 
Total params: 3114 (12.16 KB)
Trainable params: 3114 (12.16 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [4]:
# Train the model
from tensorflow.keras.callbacks import ModelCheckpoint
checkpointer=ModelCheckpoint(filepath='Predictor_models/audio_predictor_drums.hdf5',verbose=1,save_best_only=True)
history = model.fit(X_train, y_train, epochs=500, batch_size=32, validation_split=0.1, callbacks=checkpointer)

Epoch 1/500


1/2 [==============>...............] - ETA: 0s - loss: 429.1605 - mae: 14.7982
Epoch 1: val_loss improved from inf to 217.12227, saving model to Predictor_models\audio_predictor_drums.hdf5
2/2 [==============================] - 1s 175ms/step - loss: 427.1457 - mae: 14.7800 - val_loss: 217.1223 - val_mae: 11.8608
Epoch 2/500
1/2 [==============>...............] - ETA: 0s - loss: 423.9587 - mae: 14.8425
Epoch 2: val_loss improved from 217.12227 to 198.18845, saving model to Predictor_models\audio_predictor_drums.hdf5
2/2 [==============================] - 0s 47ms/step - loss: 394.3828 - mae: 14.2429 - val_loss: 198.1884 - val_mae: 11.3444
Epoch 3/500
1/2 [==============>...............] - ETA: 0s - loss: 362.4548 - mae: 13.7851
Epoch 3: val_loss improved from 198.18845 to 180.79099, saving model to Predictor_models\audio_predictor_drums.hdf5
2/2 [==============================] - 0s 56ms/step - loss: 363.0790 - mae: 13.7146 - val_loss: 180.7910 - val_mae: 10.8363
Epoch 4/50

C:\Users\Sumanth\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/2 [==============>...............] - ETA: 0s - loss: 300.2980 - mae: 12.5830
Epoch 5: val_loss improved from 164.99562 to 150.37505, saving model to Predictor_models\audio_predictor_drums.hdf5
2/2 [==============================] - 0s 47ms/step - loss: 308.7690 - mae: 12.7360 - val_loss: 150.3750 - val_mae: 9.8447
Epoch 6/500
1/2 [==============>...............] - ETA: 0s - loss: 276.1144 - mae: 12.0825
Epoch 6: val_loss improved from 150.37505 to 137.23705, saving model to Predictor_models\audio_predictor_drums.hdf5
2/2 [==============================] - 0s 47ms/step - loss: 284.0761 - mae: 12.2615 - val_loss: 137.2370 - val_mae: 9.3423
Epoch 7/500
1/2 [==============>...............] - ETA: 0s - loss: 279.1167 - mae: 12.1854
Epoch 7: val_loss improved from 137.23705 to 125.32180, saving model to Predictor_models\audio_predictor_drums.hdf5
2/2 [==============================] - 0s 48ms/step - loss: 262.3364 - mae: 11.8107 - val_loss: 125.3218 - val_mae: 8.8531
Epoch 8/500
1/2 [=====

In [7]:
# Evaluate the model on the test data
from sklearn.metrics import r2_score
loss, mae = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)

# Calculate R-squared (Coefficient of Determination)
r_squared = r2_score(y_test, y_pred)
print(f"Test Loss: {loss}")
print(f"Test Mean Absolute Error: {mae}")
print(f"r2_score:{r_squared}")

1/1 [==============================] - 0s 432ms/step
Test Loss: 7.28167724609375
Test Mean Absolute Error: 2.190002202987671
r2_score:0.943326895132256
